<a href="https://colab.research.google.com/github/Techsolutions2024/src/blob/main/ppeyolov11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mugheesahmad_sh17_dataset_for_ppe_detection_path = kagglehub.dataset_download('mugheesahmad/sh17-dataset-for-ppe-detection')

print('Data source import complete.')


Data source import complete.


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ultralytics import YOLO
import yaml
import os
from pathlib import Path

# Define paths
DATASET_PATH = '/kaggle/input/sh17-dataset-for-ppe-detection'
OUTPUT_PATH = '/kaggle/working/runs'

In [ ]:
class_names = {
11: 'hand',
0: 'person',
12: 'head',
3: 'face',
1: 'ear',
7: 'tool',
14: 'shoes',
9: 'gloves',
8: 'glasses',
10: 'helmet',
6: 'foot',
5: 'face-mask-medical',
16: 'safety-vest',
2: 'ear-mufs',
15: 'safety-suit',
13: 'medical-suit',
4: 'faceguard'
}

In [ ]:
data_config = {
    'path': DATASET_PATH,
    'train': os.path.join(DATASET_PATH, 'images'),
    'val': os.path.join(DATASET_PATH, 'images'),
    'test': os.path.join(DATASET_PATH, 'images'),

    'names': class_names,
    'nc': len(class_names),

    'train_list': os.path.join(DATASET_PATH, 'train_files.txt'),
    'val_list': os.path.join(DATASET_PATH, 'val_files.txt')
}

with open('data.yaml', 'w') as f:
    yaml.safe_dump(data_config, f, sort_keys=False)

print("Created data.yaml with configuration:")
print(yaml.safe_dump(data_config, sort_keys=False))

In [ ]:
model = YOLO('yolo11n.pt')

train_args = dict(
    data='data.yaml',
    epochs=50,
    imgsz=416,
    batch=16,
    workers=2,

    augment=True,
    degrees=5.0,
    translate=0.1,
    scale=0.3,
    fliplr=0.5,
    mosaic=0.8,
    mixup=0.05,

    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=1.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    cache=True,
    save=True,
    save_period=10,
    patience=15,

    device='0',
    project='/kaggle/working',
    name='ppe_detection_kaggle',

    exist_ok=True,
    deterministic=True
)

import os
os.environ['YOLO_CACHE_DIR'] = '/kaggle/working/.cache'

In [ ]:
results = model.train(**train_args)

print("\nTraining completed!")

In [ ]:
print("\n=== Detailed Performance Metrics ===")
print(f"\nMean Average Precision (mAP):")
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize detections
def visualize_detections(model, image_path, conf_thresh=0.25):
    # Run inference
    results = model.predict(image_path, conf=conf_thresh)[0]

    # Get the image
    img = results.orig_img

    # Create figure and axis
    plt.figure(figsize=(12, 8))
    plt.imshow(img)

    # Plot boxes
    boxes = results.boxes
    for box in boxes:
        # Get coordinates
        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Convert tensor to list
        conf = box.conf[0].item()  # Get confidence as a scalar
        cls = int(box.cls[0].item())  # Get class as an integer

        # Create rectangle
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
        plt.gca().add_patch(rect)

        # Add label with confidence
        class_name = model.names[cls]
        label = f'{class_name} {conf:.2f}'
        plt.text(x1, y1 - 5, label, color='red', fontsize=10, backgroundcolor='white')

    plt.axis('off')
    plt.show()

# Visualize multiple test images
test_images = [
    '/kaggle/input/sh17-dataset-for-ppe-detection/images/construction-site-build-construction-work-159306.jpeg',
    '/kaggle/input/sh17-dataset-for-ppe-detection/images/pexels-photo-10040003.jpeg',
    '/kaggle/input/sh17-dataset-for-ppe-detection/images/pexels-photo-10202856.jpeg'
]

for img_path in test_images:
    visualize_detections(model, img_path)